In [1]:
import time
import subprocess as sp
import requests

In [2]:
ports = [i for i in range(3001,3003)]
servers = []

In [49]:
for port in ports:
    s = sp.Popen("python server.py "+str(port), shell=True)
    
    servers.append(s)

In [50]:
port = '3001'
url = 'http://127.0.0.1:'+port+'/replicate'
myobj = {'port': '3002'}

x = requests.post(url, data = myobj)

In [42]:
for s in servers:
    s.terminate()

In [1]:
from flask import Flask, request, session, jsonify
import sys
import requests
import multiprocessing as mp

from flask_session import Session


def create_app():
    app = Flask(__name__)
    
    sess = Session()
    app.config['SESSION_TYPE'] = 'filesystem'
    app.secret_key = b'_5#y2L"F4Q8z\n\xec]/'
    sess.init_app(app)

    @app.route("/get", methods=['get'])
    def get():
        db = session.get('db')
        if not db:
            session['db'] = [0]
            db = session['db']
        print(session)
        return ''.join(str(a)+", " for a in db)
    
    
    @app.route("/set", methods=['post'])
    def set():
        if len(request.form) == 0:
            return "400: Bad Request"
        item = request.form["item"]
        print(session)
        db = session.get('db')
        print(db)
        db.append(item)
        session['db'] = db
        return "200: OK" 
    
    @app.route("/hello", methods=['get'])
    def hello():
        print(session)
        return "Hello"
    return app


    #app.run(debug=False, host='0.0.0.0', port=port,use_reloader=False,threaded=True)
    
def make_leader(app): 
    @app.route("/replicate", methods=['post','get'])
    def replicate():
        ''' send contents to another node '''

        if len(request.form) == 0:
            return "400: Bad Request"
        port = request.form["port"]
        url="http://127.0.0.1:"+port+"/store"

        data = {'items': ''.join(str(a)+", " for a in g.db)}

        x = requests.post(url, data=data)

        return x.text

    return app


def make_follower(app):
    @app.route("/store", methods=['post'])
    def store():
        ''' receive db contents from another node '''
        if len(request.form) == 0:
            return "400: Bad Request"
        items = request.form["items"]   
        items = items[:-2]
        #return items
        g.db = [int(a.strip()) for a in items.split(',')]

        return ''.join(str(a)+", " for a in g.db)
    
    


In [ ]:
app = create_app()
leader = make_leader(app)
leader.run(debug=False, host='0.0.0.0', port=9997,use_reloader=False,threaded=True)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:9997/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Jun/2020 15:33:59] "GET /get HTTP/1.1" 200 -


<FileSystemSession {'_permanent': True, 'db': [0]}>


127.0.0.1 - - [07/Jun/2020 15:34:02] "GET /hello HTTP/1.1" 200 -


<FileSystemSession {'_permanent': True}>


[2020-06-07 15:34:09,477] ERROR in app: Exception on /set [POST]
Traceback (most recent call last):
  File "/home/gov/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/gov/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/gov/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/gov/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/gov/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/gov/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-1-8ec80c816d78>", line 35, in set

<FileSystemSession {'_permanent': True}>
None


In [6]:
servers = []
for i in range(3):
    p = mp.Process(target=create_app, args=(3000+i,))
    p.start()
    servers.append(p)

In [7]:
app = servers[0]
type(app)



multiprocessing.context.Process

In [ ]:
!sudo pip3 install flask.ext

Password: